# Attacking AES Without A Trigger - Using an Edge Trigger

**This notebook can only be completed using the CWlite with the modified firmware containing the Edge Trigger module.**

As usual, we'll start by setting up the firmware for the target.

If you want to *totally* remove the trigger, navigate to `../../chipwhisperer/hardware/victims/firmware/simpleserial-aes` and open `simpleserial-aes.c` in your favourite text editor. Find the following code and comment out `trigger_high();` and `trigger_low();`:

```C
uint8_t get_pt(uint8_t* pt)
{
	trigger_high();
	aes_indep_enc(pt); /* encrypting the data block */
	trigger_low();
	simpleserial_put('r', 16, pt);
	return 0x00;
}
```

Then build the firmware:

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'AVRCRYPTOLIB'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

## Setup and Trace Analysis

In [ ]:
def load_bitstream(bitstream_file):
    %run "../Setup_Scripts/Setup_Generic.ipynb"
    scope.scopetype.cwFirmwareConfig[0xACE2].loader.setFPGAMode("debug")
    scope.scopetype.cwFirmwareConfig[0xACE2].loader._bsLoc = bitstream_file
    scope.scopetype.cwFirmwareConfig[0xACE2].loader.save_bsLoc()
    print("Mode: " + str(scope.scopetype.cwFirmwareConfig[0xACE2].loader._release_mode))
    print("Mode: " + str(scope.scopetype.cwFirmwareConfig[0xACE2].loader.fpga_bitstream()))
    input("powercycle done?")
    %run "../Setup_Scripts/Setup_Generic.ipynb"
    
load_bitstream("../../hardware/capture/chipwhisperer-lite/cwlite_interface_ec_256.bit")

And program the device:

In [ ]:
fw_path = '../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex'.format(PLATFORM)
cw.program_target(scope, prog, fw_path)

Next, let's get into what stream mode is. Essentially, stream mode allows the ChipWhisperer software to continuously read power measurements back from the scope enabling extremely long captures. In theory, there's no limit to how long these captures can be, but keep in mind very long wave (in the millions of samples) can easily outstrip your computer's memory. Even if you can store the wave itself, doing anything else with the wave may cause memory errors.

Stream mode does have one major limit though: sample rate. With stream mode enabled, the Pro can only record traces at a sample rate of less than 10MHz. Our typical 4\*7.37MHz ADC clock easily outpaces that, so we'll need to change our clocksource to be 7.37MHz instead:

In [ ]:
scope.clock.adc_src = "clkgen_x1"

We'll need to set our regular trigger to be `"low"` so that the scope will immediately trigger and start capturing after being armed:

In [ ]:
scope.adc.basic_mode = "low"
scope.trigger.module = "basic"

In [ ]:
scope.adc.samples=20000

That's it! We can now capture a trace.

In [ ]:
scope.gain.gain = 40

In [ ]:
%run "../Helper_Scripts/plot.ipynb"
plot = real_time_plot(plot_len=20000)

In [ ]:
from tqdm import tnrange
ktp = cw.ktp.Basic()

key, text = ktp.next()

for i in tnrange(10, desc='Capturing traces'):    
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    plot.send(trace)

## Configure the Edge Trigger module

In [ ]:
scope.EC.window_size = 1
scope.EC.threshold = 0.42

scope.EC.hold_cycles = 1
scope.EC.edge_num = 1
scope.EC.edge_type = "rising_edge"

scope.trigger.module = "EC"
scope.adc.basic_mode = "rising_edge"
scope.adc.presamples=400

In [ ]:
scope.EC.start()
scope.EC.check_status()

In [ ]:
scope.trigger

In [ ]:
scope.adc

## Test if trigger works

In [ ]:
from tqdm import tnrange
project = cw.create_project("projects/SAD_test", overwrite = True)
for i in tnrange(3, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)

## Capture traces using trigger

In [ ]:
%run "../Helper_Scripts/plot.ipynb"
plot = real_time_plot(plot_len=20000)

In [ ]:
from tqdm import tnrange
ktp = cw.ktp.Basic()

key, text = ktp.next()

for i in tnrange(10, desc='Capturing traces'):    
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    plot.send(trace)